In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

sk


In [2]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [3]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '아메리카노',
              'menu_number': 1,
              'source': '../../data/cafe_menu_data.txt'},
 'page_content': '1. 아메리카노\n'
                 '   • 가격: ₩4,500\n'
                 '   • 주요 원료: 에스프레소, 뜨거운 물\n'
                 '   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 '
                 '잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '카페라떼',
              'menu_number': 2,
              'source': '../../data/cafe_menu_data.txt'},
 'page_content': '2. 카페라떼\n'
                 '   • 가격: ₩5,500\n'
                 '   • 주요 원료: 에스프레소, 스팀 밀크\n'
                 '   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings


embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
cafe_db.save_local("../../db/cafe_db")


# Retriever 생성
menu_retriever = cafe_db.as_retriever(
    search_kwargs={'k': 4},
)


In [5]:
from typing import List
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from langchain_community.document_loaders import WikipediaLoader
from langchain_openai import ChatOpenAI
# Tool 정의 
# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs
# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

# menu db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "../../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from an encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]


tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

C:\Users\user\AppData\Local\Temp\ipykernel_11824\3174128721.py:36: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [6]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("%"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)         

    print("tool_msgs: \n") 
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [7]:

# 체인 실행
response = cafe_menu_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")

# 응답 출력 
print(response.content)

('db_search_cafe_func: \n'
 "{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': "
 "'call_8ssKCSmyXW3bFOFnutcmzdFb', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('wiki_summary: \n'
 "{'name': 'wiki_summary', 'args': {'query': '아메리카노'}, 'id': "
 "'call_glJie5Gf0GMNjhwUA3dFNSvl', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_cafe_func'
'wiki_summary'
----------------------------------------------------------------------------------------------------
아메리카노는 에스프레소에 뜨거운 물을 추가하여 만드는 커피 음료입니다. 이 음료는 에스프레소의 진한 맛과 향을 유지하면서 부드럽고 덜 쓴 맛을 즐길 수 있는 특징이 있습니다. 아메리카노는 기본 에스프레소와 비슷하지만, 물을 추가하여 농도를 조절하는 것이 특징입니다. 

한국의 카페에서는 아메리카노의 가격이 대개 4,500원에서 7,000원 사이입니다. 여러 카페에서의 가격은 다음과 같습니다:

- **아메리카노**: 4,500원
- **스페셜 아메리카노**: 4,500원
- **카페라떼**: 5,000원
- **플랫화이트**: 7,000원

이 음료는 일반적으로 커피콩의 원산지나 추출 방식에 따라 맛이 다양하게 변화할 수

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요", name="example_user"),
    AIMessage("메뉴 검색과 위키피디아 검색을 진행하겠습니다", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노: 가격 ₩4,500, 에스프레소와 뜨거운 물 사용, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피", tool_call_id="1"),    
    AIMessage("아메리카노의 가격은 ₩4,500이며, 에스프레소와 뜨거운 물 사용, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "아메리카노", "k": 1}, "id": "2"}]),
    ToolMessage("아메리카노(Americano)는 에스프레소를 뜨거운 물로 농도를 희석하여 마시는 커피 음료의 한 종류이다. \
                그 농도는 일반적인 드립 커피와 비슷하지만 풍미(風味)는 다르다. 아메리카노의 농도는 에스프레소의 '샷' 수와, 더해지는 물의 양에 따라 달라진다.", tool_call_id="2"),
    AIMessage("아메리카노(₩4,500)는 에스프레소를 뜨거운 물로 진한 에스프레소에 농도를 희석하여 마시는 블랙 커피의 한 종류이다. \
             그 농도는 일반적인 드립 커피와 비슷하지만 풍미(風味)는 다르다. 아메리카노의 농도는 에스프레소의 '샷' 수와, 더해지는 물의 양에 따라 달라진다.", name="example_assistant"),
]

system = """You are an AI assistant providing cafe menu information and general food/drink knowledge.
For information about the cafe's menu, use the db_search_cafe_func tool.
For other general information, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = fewshot_search_chain.invoke(query)
# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'tavily_search_func', 'args': {'query': '카페라떼와 어울리는 디저트'}, 'id': 'call_zOvJOioGPUo6YoDjDfjibvta', 'type': 'tool_call'}
{'name': 'wiki_summary', 'args': {'query': '라떼'}, 'id': 'call_aKtDo49tINnXBD9jgDOgLxq5', 'type': 'tool_call'}


In [9]:
# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_cafe_func', 'args': {'query': '카페라떼 디저트 추천'}, 'id': 'call_aeOasgA7Tnu6d0PGNBhWN1RU', 'type': 'tool_call'}
{'name': 'wiki_summary', 'args': {'query': '카페라떼'}, 'id': 'call_2mjhLCEftv9JizGwHhBvj9nE', 'type': 'tool_call'}


In [10]:
from datetime import datetime

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])



# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools


In [11]:
# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_cafe_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "카페라떼 그리고 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '카페라떼'}, 'id': 'call_T2c0D5fcKaUIdXTsFgB6yx2k', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '카페라떼', 'k': 1}, 'id': 'call_KmLotCuMRMcFYiNXu9VFaO9P', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='65985640-144e-4ca3-ba36-2c401d26a3cb', metadata={'source': '../../data/cafe_menu_data.txt', 'menu_number': 4, 'menu_name': '바닐라 라떼'}, page_content='4. 바닐라 라떼\\n   • 가격: ₩6,000\\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.'), Document(id='ec8dc692-f4ed-4a4b-9ba4-3e27de59ee24', metadata={'source': '../../data/cafe_menu_data.txt', 'menu_number': 2, 'menu_name': '카페라떼'}, page_cont